# Crea un nuevo archivo faa, donde mapea a que organismo pertenece cada funcion detectada por Prokka.
## Importar librerias

In [ ]:
import pandas as pd
from Bio import SeqIO

## Cargar rutas de los archivos de prokka y de kraken

In [ ]:
ruta_prokka="/files2/camda2024/gut/prokka/"
ruta_kraken="/files2/camda2024/gut/taxonomy/assembly/kraken/"
ruta_outputs="/files2/camda2024/gut/taxonomy/assembly/reports/"
ruta_salida="/files2/camda2024/gut/"

In [2]:
%%bash
ls /files2/camda2024/gut/prokka/

ERR209069
ERR209070
ERR209071
ERR209072
ERR209082
ERR209083
ERR209084
ERR209085
ERR209088
ERR209096
ERR209118
ERR209160
ERR209162
ERR209163
ERR209164
ERR209175
ERR209187
ERR209223
ERR209225
ERR209226
ERR209246
ERR209247
ERR209248
ERR209266
ERR209286
ERR209287
ERR209296
ERR209298
ERR209299
ERR209300
ERR209310
ERR209312
ERR209336
ERR209339
ERR209343
ERR209354
ERR209371
ERR209373
ERR209375
ERR209376
ERR209378
ERR209379
ERR209380
ERR209403
ERR209404
ERR209405
ERR209415
ERR209431
ERR209432
ERR209433
ERR209434
ERR209439
ERR209440
ERR209441
ERR209442
ERR209448
ERR209470
ERR209487
ERR209519
ERR209520
ERR209521
ERR209648
ERR209653
ERR209667
ERR209671
ERR209674
ERR209679
ERR209684
ERR209685
ERR209692
ERR209693
ERR209694
ERR209696
ERR209697
ERR209700
ERR209701
ERR209706
ERR209707
ERR209710
ERR209711
ERR209716
ERR209720
ERR209722
ERR209725
ERR209726
ERR209731
ERR209732
ERR209740
ERR209745
ERR209746
ERR209748
ERR209749
ERR209764
ERR209782
ERR209799
ERR209810
ERR209811
ERR209812
ERR209813
ERR209846


## Generar un diccionadio de id de los contigs del emsamble y id de prokka y a que organismo fue asignado.

In [ ]:
# Cargar el archivo GFF de Prokka
prokka_df = pd.read_csv(ruta_prokka+"SRR5983481/SRR5983481_anotacion.gff", sep="\t", comment="#", header=None)
prokka_df.columns = ["contig", "source", "type", "start", "end", "score", "strand", "phase", "attributes"]

# Extraer IDs de contig y de gen/proteína
prokka_df['contig_id'] = prokka_df['attributes'].apply(lambda x: x.split(';')[0].split('=')[1])

# Función para extraer el valor de 'product' de la cadena de 'attributes'
def extract_product(attributes):
    # Dividir la cadena por ';' para separar cada par clave=valor
    attributes_list = attributes.split(';')
    # Iterar sobre cada par
    for attr in attributes_list:
        if attr.startswith('product='):
            # Extraer y retornar el valor después de 'product='
            return attr.split('=')[1]
    return "Unknown"  # Retornar "Unknown" si 'product' no se encuentra

# Aplicar la función a la columna 'attributes' y crear la nueva columna 'product'
prokka_df['product'] = prokka_df['attributes'].apply(extract_product)

# Cargar el archivo de salida de Kraken
kraken_df = pd.read_csv(ruta_kraken + "SRR5983481.kraken", sep="\t", header=None)
kraken_df.columns = ["classification", "seq_id", "tax_id", "seq_length", "taxonomic_assignment"]

# Cargar el archivo de mapeo de OTUs
otu_df = pd.read_csv(ruta_outputs+"SRR5983481.report", sep="\t", header=None)
otu_df.columns = ["percentage_of_total", "num_seqs_at_taxon", "num_seqs_at_taxon_incl_subtaxa", "rank_code", "tax_id", "taxon_name"]

# Mapear tax_id de Kraken a Prokka
prokka_df = prokka_df.merge(kraken_df[['seq_id', 'tax_id']], left_on='contig', right_on='seq_id', how='left')
prokka_df.drop(columns=['seq_id'], inplace=True)  # Eliminar la columna duplicada de seq_id

# Mapear taxon_name usando tax_id
prokka_df = prokka_df.merge(otu_df[['tax_id', 'taxon_name']], on='tax_id', how='left')

## Genera un nuevo arhivo faa con los nombre de organimos que fue asigando.

In [ ]:
# Agrupar por 'contig_id' y combinar 'product' y 'taxon_name'
prokka_df['combined'] = prokka_df['product'] + " ("+ prokka_df['taxon_name'] + ")"
grouped = prokka_df.groupby('contig_id')['combined'].apply('; '.join).reset_index()

# Crear diccionario
id_map = pd.Series(grouped.combined.values, index=grouped.contig_id).to_dict()

# Modificar descripciones en FASTA
def modify_fasta_description(records, id_map):
    for record in records:
        record_id = record.id
        description = id_map.get(record_id, "Unknown")
        record.description = f"{record_id} {description}"
        yield record


# Leer el archivo FASTA original
input_fasta_path = ruta_prokka+"SRR5983481/SRR5983481.faa"
records = list(SeqIO.parse(input_fasta_path, "fasta"))

# Modificar los identificadores
modified_records = modify_fasta_description(records, id_map)

# Guardar el archivo FASTA modificado
output_fasta_path = ruta_prokka+"SRR5983481/SRR5983481_modified.faa"
SeqIO.write(modified_records, output_fasta_path, "fasta")